In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns

So, the first thing that we want to do is a little background research into what we are getting ourselves into. It appears to be a dataset of 115 first year engineering undergrads participating in an expiriment. This expiriment is designed to track the activities of the participants during their sessions within an eductional suite.

It appears that the majority of the data exists within the `Data/Processes/` directory. This directory contains 6 sub-directories for each of the 6 sessions tracked. Within these `Session` folders. One of the first tasks is going to be to read in all of these data files, but before that, we must create labels for the header row. The headers are found in `features_info.txt`. 

In [2]:
header_list = ['session', 'student_id', 'exercise', 'activity', 'start_time', 
               'end_time', 'idle_time','mouse_wheel', 'mouse_wheel_click',
               'left_click','right_click','mouse_movement','keystroke']

In [3]:
allFiles = glob.glob('Data/Processes/*/*') 
frame = pd.DataFrame() 
list_ = [] 

for file_ in allFiles: 
    df = pd.read_csv(file_)
    df.columns = header_list
    list_.append(df)
frame = pd.concat(list_) 


In [4]:
frame.head()

,session,student_id,exercise,activity,start_time,end_time,idle_time,mouse_wheel,mouse_wheel_click,left_click,right_click,mouse_movement,keystroke
0,1,1,Es,Aulaweb,2.10.2014 11:25:35,2.10.2014 11:25:42,218,0,0,4,0,397,0
1,1,1,Es,Blank,2.10.2014 11:25:43,2.10.2014 11:25:43,0,0,0,0,0,59,0
2,1,1,Es,Deeds,2.10.2014 11:25:44,2.10.2014 11:26:17,154117,6,0,8,0,1581,4
3,1,1,Es,Other,2.10.2014 11:26:18,2.10.2014 11:26:18,0,0,0,2,0,103,0
4,1,1,Es,Other,2.10.2014 11:26:19,2.10.2014 11:26:27,460,0,0,4,0,424,8


In [5]:
frame.describe()

,session,student_id,idle_time,mouse_wheel,mouse_wheel_click,left_click,right_click,mouse_movement,keystroke
count,229798.000000,229798.000000,2.297980e+05,229798.000000,229798.000000,229798.000000,229798.000000,229798.000000,229798.000000
mean,3.695058,53.626233,-1.852064e+09,2.749741,0.005457,7.083112,0.336870,415.060222,6.294045
std,1.770348,31.542913,4.502500e+11,27.169832,0.223620,21.567963,3.020974,1273.675902,51.017414
min,1.000000,1.000000,-2.059092e+14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,27.000000,0.000000e+00,0.000000,0.000000,2.000000,0.000000,62.000000,0.000000
50%,4.000000,53.000000,8.000000e+01,0.000000,0.000000,2.000000,0.000000,138.000000,0.000000
75%,5.000000,81.000000,5.779000e+03,0.000000,0.000000,5.000000,0.000000,336.000000,0.000000
max,6.000000,115.000000,7.244736e+09,2904.000000,60.000000,1096.000000,168.000000,85949.000000,4754.000000


Whoops! Looks like the `session` and `student_id` columns are continuous. Let's go ahead and factorize those. Also, while we are at it, let's take a look at why data types each of the columns are. I am suspicious about the `start_time` and `end_time` features.

In [6]:
for col in ['session', 'student_id']:
    frame[col] = frame[col].astype('category')
    
for col in ['start_time', 'end_time']:
    frame[col] =  pd.to_datetime(frame[col], format=' %d.%m.%Y %H:%M:%S')

#frame['start_time'] = pd.to_datetime(frame['start_time'], format=' %d.%m.%Y %H:%M:%S')
#frame['end_time'] = pd.to_datetime(frame['end_time'], format=' %d.%m.%Y %H:%M:%S')

print(frame.columns.to_series().groupby(frame.dtypes).groups)


{dtype('<M8[ns]'): ['start_time', 'end_time'], category: ['session', 'student_id'], dtype('int64'): ['idle_time', 'mouse_wheel', 'mouse_wheel_click', 'left_click', 'right_click', 'mouse_movement', 'keystroke'], dtype('O'): ['exercise', 'activity']}


In [7]:
frame.describe()


,idle_time,mouse_wheel,mouse_wheel_click,left_click,right_click,mouse_movement,keystroke
count,2.297980e+05,229798.000000,229798.000000,229798.000000,229798.000000,229798.000000,229798.000000
mean,-1.852064e+09,2.749741,0.005457,7.083112,0.336870,415.060222,6.294045
std,4.502500e+11,27.169832,0.223620,21.567963,3.020974,1273.675902,51.017414
min,-2.059092e+14,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000,0.000000,2.000000,0.000000,62.000000,0.000000
50%,8.000000e+01,0.000000,0.000000,2.000000,0.000000,138.000000,0.000000
75%,5.779000e+03,0.000000,0.000000,5.000000,0.000000,336.000000,0.000000
max,7.244736e+09,2904.000000,60.000000,1096.000000,168.000000,85949.000000,4754.000000


Okay, so it looks like we can get some descriptive statistics on the continuous variables. How about looking at the difference between `end_time` and `start_time`. This might be an interesting variable to track performance later on.

In [8]:
frame['time_delta'] = frame['end_time']-frame['start_time']
frame['time_delta'].describe()

count                    229798
mean     0 days 00:00:15.870281
std      0 days 00:01:01.079216
min           -1 days +23:49:48
25%             0 days 00:00:00
50%             0 days 00:00:02
75%             0 days 00:00:09
max             0 days 01:04:21
Name: time_delta, dtype: object

Look at that minimum time! Surely that has to be an input error as negative time is not possible. Hopefully this doesn't happen a lot. Let's find the columns in the `frame` dataframe to see if it is a common problem. 

In [9]:
frame[frame['start_time']>frame['end_time']]

,session,student_id,exercise,activity,start_time,end_time,idle_time,mouse_wheel,mouse_wheel_click,left_click,right_click,mouse_movement,keystroke,time_delta
562,4,89,Es_4_5,Diagram,2014-11-13 13:50:40,2014-11-13 13:40:28,30681,0,0,42,0,636,0,-1 days +23:49:48


Looks like it isn't that big of a deal. Only one row in the dataframe appears to have an incorrectly inputed times (well, at least for those errors that can be ruled out by physical impossibility). We will want take this out if doing an analysis that includes time as a variable, however, there are other parts of data that may be pertinent outside of the scope of time. For that reason, we shall keep this in the final file.

In [78]:
#frame.to_csv('sessions.csv')

In [40]:
sheets = pd.ExcelFile('Data/final_grades.xlsx')
df1 = sheets.parse(0)
df2 = sheets.parse(1)

In [44]:
frames = [df1,df2]
final_grades = pd.concat(frames)

In [45]:
final_grades

,Student ID,ES 1.1 (2 points),ES 1.2 (3 points),ES 2.1 (2 points),ES 2.2 (3 points),ES 3.1 (1 points),ES 3.2 (2 points),ES 3.3 (2 points),ES 3.4 (2 points),ES 3.5 (3 points),ES 4.1 (15 points),ES 4.2 (10 points),ES 5.1 (2 points),ES 5.2 (10 points),ES 5.3 (3 points),ES 6.1 (25 points),ES 6.2 (15 points),TOTAL (100 points)
0,3,2.0,3.0,1.0,2.0,1.0,2,2.0,2,3.0,15.0,10.0,1.0,5.0,3.0,18.0,15.0,85.0
1,6,2.0,3.0,2.0,3.0,1.0,2,2.0,0,3.0,15.0,7.0,2.0,9.0,3.0,13.0,15.0,82.0
2,7,2.0,3.0,1.0,1.5,1.0,2,0.0,0,3.0,5.0,4.0,0.0,0.0,3.0,17.0,10.0,52.5
3,10,2.0,3.0,2.0,1.5,1.0,2,0.0,2,3.0,11.0,1.0,2.0,10.0,1.5,7.0,10.0,59.0
4,13,2.0,3.0,2.0,1.5,1.0,2,2.0,2,3.0,14.5,10.0,2.0,2.0,3.0,25.0,15.0,90.0
5,15,2.0,3.0,1.0,2.0,1.0,2,2.0,2,3.0,15.0,10.0,2.0,4.0,1.5,2.0,15.0,67.5
6,16,2.0,3.0,1.0,0.0,1.0,2,2.0,2,3.0,3.0,9.0,1.0,0.0,3.0,20.0,15.0,67.0
7,17,2.0,3.0,1.0,2.0,1.0,2,2.0,2,3.0,15.0,10.0,2.0,10.0,3.0,24.0,15.0,97.0
8,18,1.0,3.0,2.0,3.0,1.0,2,2.0,2,3.0,15.0,7.0,2.0,2.5,1.5,5.0,10.0,62.0
9,20,2.0,3.0,2.0,0.0,1.0,2,2.0,2,1.5,15.0,10.0,0.0,0.0,3.0,5.0,10.0,58.5
